In [10]:
import pandas as pd
import numpy as np

In [11]:
class DataFrame:
    def selecionar_atributos(self, data_frame, lista_atributos):
        data_frame = data_frame[lista_atributos]
        return data_frame
    
    def selecionar_cursos(self, data_frame, lista_atributos, indice_area, codigos_cursos_de_computacao):
        data_frame = data_frame[data_frame[lista_atributos[indice_area]].isin(codigos_cursos_de_computacao)]
        return data_frame
    
    def ordenar_por_codigo_curso(self, data_frame, lista_atributos, indice_curso):
        data_frame = data_frame.sort_values(lista_atributos[indice_curso])
        data_frame = data_frame.reset_index(drop=True)
        return data_frame
    
    def converter_valores_numericos_para_int(self, data_frame):
        data_frame = data_frame.astype('int')
        return data_frame
    
    def criar_coluna_numero_faltantes(self, data_frame, lista_atributos, indice_inscritos, indice_participantes):
        data_frame["Numero_Faltantes"] = data_frame[lista_atributos[indice_inscritos]] - data_frame[lista_atributos[indice_participantes]]
        return data_frame
    
    def renomear_colunas(self, data_frame, lista_atributos, indice_curso, indice_area, indice_inscritos, indice_participantes):
        data_frame.rename(columns = {lista_atributos[indice_curso]:'Codigo_do_Curso', lista_atributos[indice_area]: 'Curso', lista_atributos[indice_inscritos]: 'Numero_Inscritos', lista_atributos[indice_participantes]: 'Numero_Participantes'}, inplace = True)
        return data_frame
    
    def excluir_linhas_invalidas(self, data_frame):
        linhas_invalidas = self.obter_lista_linhas_invalidas(data_frame)
        data_frame = data_frame.drop(linhas_invalidas)
        data_frame = data_frame.reset_index(drop=True)
        return data_frame
        
    def obter_lista_linhas_invalidas(self, data_frame):
        linhas_invalidas = []
        
        for index, linha in data_frame.iterrows():
            if linha["Numero_Participantes"] <= 1:
                linhas_invalidas.append(index)
                
        return linhas_invalidas
        
class DataFrame_2021(DataFrame):
    data_frame = pd.read_csv('Conceito_ENADE/conceito_enade_2021.csv')
    lista_atributos = ["Código do Curso", "Código da Área", "Nº de Concluintes Inscritos", "Nº  de Concluintes Participantes"]
    indice_curso, indice_area, indice_inscritos, indice_participantes = 0, 1, 2, 3
    codigos_cursos_de_computacao = [72, 4004, 6409, 4005, 79, 4006]
    
#class DataFrame_2017(DataFrame):
    #data_frame = pd.read_csv('Conceito_ENADE/conceito_enade_2017.csv')
    #lista_atributos = ["Código do Curso", "Código da Área", "Nº de Concluintes Inscritos", "Nº  de Concluintes Participantes"]
    #indice_curso, indice_area, indice_inscritos, indice_participantes = 0, 1, 2, 3
    #codigos_cursos_de_computacao = [72, 4004, 6409, 4003, 4005, 79, 4006]
    
#class DataFrame_2014(DataFrame):
    #data_frame = pd.read_csv('Conceito_ENADE/conceito_enade_2014.csv')
    #lista_atributos = ["Cód. IES", "Cód Area", "Inscritos", "Participantes"]
    #indice_curso, indice_area, indice_inscritos, indice_participantes = 0, 1, 2, 3
    #codigos_cursos_de_computacao = [72, 4004, 5809, 4005, 79, 4006]
    
#class DataFrame_2011(DataFrame):
    #data_frame = pd.read_csv('Conceito_ENADE/conceito_enade_2011.csv')
    #lista_atributos = ["Código IES", "Código Área Agrupamento", "Número Concluintes Inscritos", "Número de Concluintes Participantes"]
    #indice_curso, indice_area, indice_inscritos, indice_participantes = 0, 1, 2, 3
    #codigos_cursos_de_computacao = [72, 4004, 5809, 4005, 79, 4006]
    

In [12]:
class Gerenciador_dataFrame:
    def ajustar_dataFrame(self, dataFrame):
        dataFrame.data_frame = dataFrame.selecionar_atributos(dataFrame.data_frame, dataFrame.lista_atributos)
        dataFrame.data_frame = dataFrame.selecionar_cursos(dataFrame.data_frame, dataFrame.lista_atributos, dataFrame.indice_area, dataFrame.codigos_cursos_de_computacao)
        dataFrame.data_frame = dataFrame.ordenar_por_codigo_curso(dataFrame.data_frame, dataFrame.lista_atributos, dataFrame.indice_curso)
        dataFrame.data_frame = dataFrame.converter_valores_numericos_para_int(dataFrame.data_frame)
        dataFrame.data_frame = dataFrame.criar_coluna_numero_faltantes(dataFrame.data_frame, dataFrame.lista_atributos, dataFrame.indice_inscritos, dataFrame.indice_participantes)
        dataFrame.data_frame = dataFrame.renomear_colunas(dataFrame.data_frame, dataFrame.lista_atributos, dataFrame.indice_curso, dataFrame.indice_area, dataFrame.indice_inscritos, dataFrame.indice_participantes)
        dataFrame.data_frame = dataFrame.excluir_linhas_invalidas(dataFrame.data_frame)
        return dataFrame
    
    def concatenar_data_frames(self, lista_data_frames):
        resultado = pd.concat(lista_data_frames)
        resultado = resultado.reset_index(drop=True)
        return resultado
    
    def gerar_coluna_numero_faltantes(self, df_todos_anos):
        codigo_cursos = self.criar_lista_codigo_cursos(df_todos_anos)

        lista_media_numero_faltantes = []
        
        lista_media_numero_faltantes = self.gerar_lista_media_numero_faltantes(codigo_cursos, df_todos_anos)
        
        coluna_numero_faltantes = self.converter_lista_para_series(lista_media_numero_faltantes)
        
        return coluna_numero_faltantes
    
    def criar_lista_codigo_cursos(self, df_todos_anos):
        array_codigo_cursos = df_todos_anos["Codigo_do_Curso"].unique()
        codigo_cursos = array_codigo_cursos.tolist()
        return codigo_cursos
    
    def gerar_lista_media_numero_faltantes(self, codigo_cursos, df_todos_anos):
        lista_media_numero_faltantes = []
        
        for codigo in codigo_cursos:
            subconjunto = self.obter_subconjunto_do_codigo(df_todos_anos, codigo)
            media_faltantes = subconjunto["Numero_Faltantes"].mean()
            lista_media_numero_faltantes.append(media_faltantes)
            
        return lista_media_numero_faltantes
            
    def obter_subconjunto_do_codigo(self, df_todos_anos, codigo):
        subconjunto = df_todos_anos.loc[df_todos_anos['Codigo_do_Curso'] == codigo]
        subconjunto = subconjunto.reset_index(drop=True)
        return subconjunto
    
    def converter_lista_para_series(self, lista_media_numero_faltantes):
        series_numero_faltantes = pd.Series(lista_media_numero_faltantes)
        series_numero_faltantes = series_numero_faltantes.round(0).astype(int)
        return series_numero_faltantes

In [13]:
gerenciador = Gerenciador_dataFrame()

In [14]:
df_2021 = DataFrame_2021()
#df_2017 = DataFrame_2017()
#df_2014 = DataFrame_2014()
#df_2011 = DataFrame_2011()

df_2021 = gerenciador.ajustar_dataFrame(df_2021)
#df_2017 = gerenciador.ajustar_dataFrame(df_2017)
#df_2014 = gerenciador.ajustar_dataFrame(df_2014)
#df_2011 = gerenciador.ajustar_dataFrame(df_2011)

In [15]:
df_2021.data_frame

,Codigo_do_Curso,Curso,Numero_Inscritos,Numero_Participantes,Numero_Faltantes
0,36,4004,41,36,5
1,127,4004,112,85,27
2,317,4004,99,82,17
3,513,4004,57,45,12
4,594,4004,47,43,4
...,...,...,...,...,...
1501,5001296,79,10,8,2
1502,5001383,72,15,11,4
1503,5001409,72,8,8,0
1504,5001434,6409,25,22,3


In [ ]:
df_2017.data_frame

In [ ]:
df_2014.data_frame

In [ ]:
df_2011.data_frame

In [ ]:
lista_data_frames = [df_2021.data_frame, df_2017.data_frame, df_2014.data_frame, df_2011.data_frame]

df_todos_anos = gerenciador.concatenar_data_frames(lista_data_frames)

In [ ]:
df_todos_anos.to_csv('./{}.csv'.format('df_numero_faltantes'),index=False)

len(df_todos_anos)

In [ ]:
coluna_numero_faltantes = gerenciador.gerar_coluna_numero_faltantes(df_todos_anos)

In [ ]:
coluna_numero_faltantes

In [ ]:
teste_insercao_coluna = pd.read_csv('testeTransformacaoDados.csv')

In [ ]:
teste_insercao_coluna["Numero_Evasoes"] = numero_evasoes

In [ ]:
teste_insercao_coluna